In [1]:
sc

# Load sokulee json data

In [2]:
df_sleep = spark.read.json('/home/datascience/sokulee/*/*_sleep.json')
df_steps = spark.read.json('/home/datascience/sokulee/*/*_steps.json')
df_heart = spark.read.json('/home/datascience/sokulee/*/*_heart.json')

In [3]:
df_sleep.printSchema()

root
 |-- sleep: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- awakeCount: long (nullable = true)
 |    |    |-- awakeDuration: long (nullable = true)
 |    |    |-- awakeningsCount: long (nullable = true)
 |    |    |-- dateOfSleep: string (nullable = true)
 |    |    |-- duration: long (nullable = true)
 |    |    |-- efficiency: long (nullable = true)
 |    |    |-- isMainSleep: boolean (nullable = true)
 |    |    |-- logId: long (nullable = true)
 |    |    |-- minuteData: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- dateTime: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- minutesAfterWakeup: long (nullable = true)
 |    |    |-- minutesAsleep: long (nullable = true)
 |    |    |-- minutesAwake: long (nullable = true)
 |    |    |-- minutesToFallAsleep: long (nullable = true)
 |    |    |-- restlessCount: long (nullable = true)
 |

In [4]:
df_steps.printSchema()

root
 |-- activities-steps: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dateTime: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- activities-steps-intraday: struct (nullable = true)
 |    |-- dataset: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- time: string (nullable = true)
 |    |    |    |-- value: long (nullable = true)
 |    |-- datasetInterval: long (nullable = true)
 |    |-- datasetType: string (nullable = true)
 |-- errors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- errorType: string (nullable = true)
 |    |    |-- message: string (nullable = true)
 |-- success: boolean (nullable = true)



In [5]:
df_heart.printSchema()

root
 |-- activities-heart: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dateTime: string (nullable = true)
 |    |    |-- value: struct (nullable = true)
 |    |    |    |-- customHeartRateZones: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- heartRateZones: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- caloriesOut: double (nullable = true)
 |    |    |    |    |    |-- max: long (nullable = true)
 |    |    |    |    |    |-- min: long (nullable = true)
 |    |    |    |    |    |-- minutes: long (nullable = true)
 |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- restingHeartRate: long (nullable = true)
 |-- activities-heart-intraday: struct (nullable = true)
 |    |-- dataset: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- time: stri

# Mean, Min, Max of data

In [6]:
from pyspark.sql.functions import *
from pyspark.sql import *

In [7]:
df_sleep.select(mean(df_sleep['summary']['totalMinutesAsleep']), min(df_sleep['summary']['totalMinutesAsleep']), max(df_sleep['summary']['totalMinutesAsleep'])).show()
df_sleep.select(mean(df_sleep['summary']['totalTimeInBed']), min(df_sleep['summary']['totalTimeInBed']), max(df_sleep['summary']['totalTimeInBed'])).show()

+----------------------------------+----------------------------------+----------------------------------+
|avg(summary['totalMinutesAsleep'])|min(summary['totalMinutesAsleep'])|max(summary['totalMinutesAsleep'])|
+----------------------------------+----------------------------------+----------------------------------+
|                306.74880239520957|                                 0|                              1199|
+----------------------------------+----------------------------------+----------------------------------+

+------------------------------+------------------------------+------------------------------+
|avg(summary['totalTimeInBed'])|min(summary['totalTimeInBed'])|max(summary['totalTimeInBed'])|
+------------------------------+------------------------------+------------------------------+
|            332.98952095808386|                             0|                          1219|
+------------------------------+------------------------------+---------------------

In [8]:
df_steps.select(mean(df_steps['activities-steps'][0]['value'])).show()

+---------------------------------+
|avg(activities-steps[0]['value'])|
+---------------------------------+
|               10478.834570834571|
+---------------------------------+



# K-means clustering with step, heart, sleep data

## Data processing

In [9]:
from pyspark.sql.types import *
import pandas as pd

In [10]:
base_path = '/home/datascience/sokulee/'
sokulee_dir = os.listdir(base_path)

data_cols = ['sleep', 'steps', 'heart']
datas = []

users_cols = ['user', 'sleep', 'steps', 'heart']
users_datas = []

for user in sokulee_dir:
    user_dir = base_path + user + '/'
    #print(user_dir)
    
    user_df_sleep = spark.read.json(user_dir + '*_sleep.json')
    user_df_steps = spark.read.json(user_dir + '*_steps.json')
    user_df_heart = spark.read.json(user_dir + '*_heart.json')
    
    print(user_dir)
    mean_of_sleep = user_df_sleep.select(mean(user_df_sleep['summary']['totalMinutesAsleep']))
    mean_of_steps = user_df_steps.select(mean(user_df_steps['activities-steps'][0]['value']))
    mean_of_heart = user_df_heart.select(
                    mean(user_df_heart['activities-heart'][0]['value']['heartRateZones'][0]['minutes'] +
                         user_df_heart['activities-heart'][0]['value']['heartRateZones'][1]['minutes'] +
                         user_df_heart['activities-heart'][0]['value']['heartRateZones'][2]['minutes'] +
                         user_df_heart['activities-heart'][0]['value']['heartRateZones'][3]['minutes']))
    
    #df.collect([mean_of_sleep.head()[0], mean_of_steps.head()[0], mean_of_heart.head()[0]])
    print(mean_of_sleep.head()[0])
    print(mean_of_steps.head()[0])
    print(mean_of_heart.head()[0])
    datas.append([mean_of_sleep.head()[0], mean_of_steps.head()[0], mean_of_heart.head()[0]])
    users_datas.append([user, mean_of_sleep.head()[0], mean_of_steps.head()[0], mean_of_heart.head()[0]])

/home/datascience/sokulee/A028/
401.3673469387755
8428.42
1173.9591836734694
/home/datascience/sokulee/A020/
295.265306122449
13525.16
1157.08
/home/datascience/sokulee/A026/
338.85714285714283
21405.82
1338.9
/home/datascience/sokulee/A053/
353.88
5784.46
1243.304347826087
/home/datascience/sokulee/A043/
360.76
19486.72
1244.28
/home/datascience/sokulee/A071/
271.52
8121.24
1055.7441860465117
/home/datascience/sokulee/A056/
295.54
10891.56
1259.72
/home/datascience/sokulee/A030/
253.48979591836735
9554.96
1147.96
/home/datascience/sokulee/A024/
214.6530612244898
15188.28
1093.62
/home/datascience/sokulee/A033/
175.0
8298.3
1155.7142857142858
/home/datascience/sokulee/A03/
376.9591836734694
7826.04
1157.32
/home/datascience/sokulee/A034/
339.88
7157.74
1129.8367346938776
/home/datascience/sokulee/A097/
186.28
5527.52
744.875
/home/datascience/sokulee/A032/
372.04
14578.32
1346.9
/home/datascience/sokulee/A045/
372.76
16718.14
1392.34
/home/datascience/sokulee/A057/
344.0
6383.9
834.285

In [11]:
data_df = pd.DataFrame(datas, columns=data_cols).round(2)
user_df = pd.DataFrame(users_datas, columns=users_cols).round(2)

In [12]:
data_df.head()

,sleep,steps,heart
0,401.37,8428.42,1173.96
1,295.27,13525.16,1157.08
2,338.86,21405.82,1338.90
3,353.88,5784.46,1243.30
4,360.76,19486.72,1244.28


In [13]:
data_df['sleep'] = data_df['sleep'].map('1:{:.2f}'.format)
data_df['steps'] = data_df['steps'].map('2:{:.2f}'.format)
data_df['heart'] = data_df['heart'].map('3:{:.2f}'.format)
data_df

,sleep,steps,heart
0,1:401.37,2:8428.42,3:1173.96
1,1:295.27,2:13525.16,3:1157.08
2,1:338.86,2:21405.82,3:1338.90
3,1:353.88,2:5784.46,3:1243.30
4,1:360.76,2:19486.72,3:1244.28
5,1:271.52,2:8121.24,3:1055.74
6,1:295.54,2:10891.56,3:1259.72
7,1:253.49,2:9554.96,3:1147.96
8,1:214.65,2:15188.28,3:1093.62
9,1:175.00,2:8298.30,3:1155.71


In [14]:
#df.to_csv('avr.csv', index=False)
data_df.to_csv('avr.txt', sep=' ', header=False)

In [15]:
from pyspark.ml.clustering import KMeans

In [16]:
# Loads data.
#dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")
''' Data format
0 1:0.0 2:0.0 3:0.0
1 1:0.1 2:0.1 3:0.1
2 1:0.2 2:0.2 3:0.2
3 1:9.0 2:9.0 3:9.0
4 1:9.1 2:9.1 3:9.1
5 1:9.2 2:9.2 3:9.2
'''
dataset = spark.read.format("libsvm").load('avr.txt')

In [17]:
# Trains a k-means model.
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 175615540.264462
Cluster Centers: 
[  309.65529412  7192.55235294  1166.25294118]
[   318.97        16904.64153846   1261.49076923]
[   295.49047619  11704.44285714   1178.70571429]


# Euclidean distance

In [18]:
user_df

,user,sleep,steps,heart
0,A028,401.37,8428.42,1173.96
1,A020,295.27,13525.16,1157.08
2,A026,338.86,21405.82,1338.90
3,A053,353.88,5784.46,1243.30
4,A043,360.76,19486.72,1244.28
5,A071,271.52,8121.24,1055.74
6,A056,295.54,10891.56,1259.72
7,A030,253.49,9554.96,1147.96
8,A024,214.65,15188.28,1093.62
9,A033,175.00,8298.30,1155.71


In [19]:
me = pd.Series(['A000', 300, 9500, 1100], index = ['user', 'sleep', 'steps', 'heart'])

In [20]:
import math

def euclidean_distance(a, b):    
    return math.sqrt( (a['sleep'] - b['sleep'])**2 + 
                      (a['steps'] - b['steps'])**2 + 
                      (a['heart'] - b['heart'])**2  )

def norm_euclidean_distance(a, b):
    return 1 / (euclidean_distance(a, b) + 1)

In [21]:
normoalized_dist = -1
nearest_user = 'A00'
for index, row in user_df.iterrows():
    dist = norm_euclidean_distance(me, row)
    if normoalized_dist < dist:
        normoalized_dist = dist
        nearest_user = row['user']

print("Nearest user is %s, normalized_dist = %.10f" %(nearest_user, normoalized_dist))

Nearest user is A030, normalized_dist = 0.0114272932
